# Properties
The `Properties.ipynb` notebook takes the scraped data obtained from Domain.com 
and goes through data cleaning stages. This includes extracting relevant information
from strings, data type casting and checking that the entries are valid. This data 
is merged with the ABS regional population data. The final output of this notebook 
is a clean dataframe that contains combined information from the ABS regional 
population data and domain.com property data, indexed by postcodes

Setting up workspace:

In [1]:
# Import relative libraries
from pyspark.sql.types import *
from pyspark.sql import SparkSession, functions as F
import re
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

# Load in Data
raw_properties = spark.read.parquet('../data/raw/property_data.parquet')
initial_instances = raw_properties.count()

your 131072x1 screen size is bogus. expect trouble
Picked up _JAVA_OPTIONS: -Xmx2048m
Picked up _JAVA_OPTIONS: -Xmx2048m
24/10/18 10:42:50 WARN Utils: Your hostname, Rachel resolves to a loopback address: 127.0.1.1; using 172.20.97.252 instead (on interface eth0)
24/10/18 10:42:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/18 10:42:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Initial Preprocessing:

In [2]:
pdf = raw_properties.toPandas()

# Delete Rows that do not contain the price in cost_text - invalid entries
valid_costs = [x if re.search(r"([\d\.]+)", x) else 0 for x in pdf['cost_text']]
pdf['cost_text'] = valid_costs
pdf = pdf[pdf['cost_text']!=0]

# Remove commas
pdf['cost'] = pdf['cost_text'].replace(',','', regex=True)

# Manual deletions 
pdf = pdf.drop(8235) # invalid cost

Creating Boolean columns to mark whether a columns cost is given per week, month or year as well as removing decimals, $ and ,

In [3]:
# Convert to Spark
sdf = spark.createDataFrame(pdf)
raw_sdf = sdf.withColumn(
    # Properties priced per week
    'Week',
    F.when(F.lower(F.col('cost_text')).contains('pw') \
           | F.lower(F.col('cost_text')).contains('p/w') \
           | F.lower(F.col('cost_text')).contains('wk') \
           | F.lower(F.col('cost_text')).contains('/w') \
           | F.lower(F.col('cost_text')).contains('week'), True).otherwise(False)
).withColumn(
    # see if property is priced per annum
    'month',
    F.when(F.lower(F.col('cost_text')).contains('month') \
        | F.lower(F.col('cost_text')).contains('per month') \
        | F.lower(F.col('cost_text')).contains('pcm'), True ).otherwise(False)
).withColumn(
    # see if property is priced per annum
    'contains_pa',
    F.when(F.lower(F.col('cost_text')).contains('p.a') \
        | F.lower(F.col('cost_text')).contains('pa') \
        | F.lower(F.col('cost_text')).contains('per annum') \
        | F.lower(F.col('cost_text')).contains('per year') \
        | F.lower(F.col('cost_text')).contains('per_annum') \
        | F.lower(F.col('cost_text')).contains('p/a'), True ).otherwise(False)
).withColumn(
    # remove $ and ,
    'cost',
    F.regexp_replace(F.col("cost"), r"[$]", "")
).withColumn(
    # removes decimal places
    'cost',
    F.regexp_replace(F.col("cost"), r"\.\d+", "")
)

Extract numeric cost value and convert to weekly cost

In [4]:
# convert to pandas
pdf = raw_sdf.toPandas()
pdf['cost'] = [(re.findall(r'\d\d+', x)) for x in pdf['cost']]

pdf['cost'] = [x[0] if (len(x)>=1) else 0 for x in pdf['cost'] ] # extract lowest estimate of price when a range is given

for i in range(len(pdf)):
    if ((pdf['month'][i] == True)&(pdf['Week'][i]==False)):
        cost = re.findall(r'\d+', pdf['cost'][i])[0]
        # Divide monthly cost by 4
        pdf['cost'][i] = int(cost)/4
    if ((pdf['month'][i] == False)&(pdf['Week'][i]==False)&((pdf['contains_pa'][i])==True)):
        cost = re.findall(r'\d+', pdf['cost'][i])[0]
        # Divide annual cost by 52
        pdf['cost'][i] = int(cost)/52

/tmp/ipykernel_9223/1662031786.py:11: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pdf['cost'][i] = int(cost)/4
/tmp/ipykernel_9223/1662031786.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Drop invalid costs, validity checking

In [5]:
pdf[(pdf['month'] == False)&(pdf['Week']==False)&((pdf['contains_pa'])==True)]

# Delete invalid costs
pdf = pdf.drop(4951)
pdf = pdf.drop(10581)

pdf[(pdf['month'] == False)&(pdf['Week']==False)&((pdf['contains_pa'])==True)] 


,url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,cost,Week,month,contains_pa
3106,https://www.domain.com.au/8-chamberlain-road-r...,3523,redcastle,"8 Chamberlain Road, Redcastle VIC 3523","$28,000 Per Year !!",None,None,0,Vacant land,538.461538,False,False,True
4900,https://www.domain.com.au/22-enterprise-way-ya...,3730,yarrawonga,"22 Enterprise Way, Yarrawonga VIC 3730","$52,000 per annum plus GST",1,1,4,New House & Land,1000.0,False,False,True
6357,https://www.domain.com.au/667-glenhuntly-road-...,3162,caulfield,"667 Glenhuntly Road, Caulfield VIC 3162","$38,000 p.a. Incl. Outgoings + GST",None,None,0,House,730.769231,False,False,True
7664,https://www.domain.com.au/12-panama-street-wol...,3750,wollert,"12 Panama Street, Wollert VIC 3750","LEASED for $33,500 PA in ONE HOUR!",4,2,2,House,644.230769,False,False,True
9720,https://www.domain.com.au/30-south-concourse-b...,3193,beaumaris,"30 South Concourse, Beaumaris VIC 3193","$39,000 p.a + Outgoings",0,1,0,Studio,750.0,False,False,True


Create Boolean column to mark if a property is furnished

In [6]:
# Convert pdf back to spark
raw_sdf = spark.createDataFrame(pdf)

raw_sdf = raw_sdf.withColumn(
    # see if property is furnished - note: this will contain partially furnished as well
    'furnished', 
    F.when(F.lower(F.col('cost_text')).contains('furnish') \
        | F.lower(F.col('cost_text')).contains('furniture'), True ).otherwise(False)
)

raw_sdf


url,postcode,suburb,name,cost_text,beds,baths,parking,property_type,cost,Week,month,contains_pa,furnished
https://www.domai...,3141,south-yarra,(Leased) 3 Yarra ...,$460,1,1,1,Apartment / Unit ...,460,false,false,false,false
https://www.domai...,3125,burwood,04/390 Burwood Hi...,"$310 per week, wi...",1,1,0,Apartment / Unit ...,310,true,false,false,false
https://www.domai...,3156,ferntree-gully,1 & 2/32 Folkston...,$800,4,3,1,House,800,false,false,false,false
https://www.domai...,3228,torquay,"1 Acacia Street, ...","$1,000 per week",4,2,2,House,1000,true,false,false,false
https://www.domai...,3200,frankston-north,1 Aleppo Crescent...,$450,3,1,1,House,450,false,false,false,false
https://www.domai...,3011,footscray,1 Bed 1 Bath/34 C...,$480 P/W,1,1,0,Apartment / Unit ...,480,true,false,false,false
https://www.domai...,3011,footscray,1 Bed 1 Bath/48 C...,$500 P/W,1,1,1,Apartment / Unit ...,500,true,false,false,false
https://www.domai...,3030,point-cook,1 Bensonhurst Par...,$570 per week,4,2,2,House,570,true,false,false,false
https://www.domai...,3047,broadmeadows,"1 Biltris Court, ...",$500,3,1,2,House,500,false,false,false,false
https://www.domai...,3149,mount-waverley,1 Birralee Street...,$625 per week,3,1,1,House,625,true,false,false,false


Cast to Correct Datatypes

In [7]:
from pyspark.sql.functions import col

pdf = raw_sdf.toPandas()

# Schema - cast datatypes
raw_sdf = raw_sdf.select(
    col('cost').cast('float'),
    col('postcode').cast('int'),
    col('suburb').cast('string'),
    col('furnished').cast('int'),
    col('property_type').cast('string'),
    col('beds').cast('int'),
    col('baths').cast('int'),
    col('parking').cast('int'),
  )

pdf=raw_sdf.toPandas()

Deleting non-zero values and removing NaN values

In [8]:
# Delete non-zero values
pdf = pdf[pdf['cost']!=0]

# Remove NaN values
pdf = pdf.dropna()
num_deletions = initial_instances - len(pdf)
print("Deleted {} instances".format(num_deletions))

Deleted 335 instances


In [9]:
final_pdf = pdf
final_sdf = spark.createDataFrame(final_pdf)

final_sdf.write.mode("overwrite").parquet("../data/curated/properties.parquet")

Validity Checking - setting the number of amenities to a range that is suitable for our demographic of renters 

In [10]:
df = spark.read.parquet('../data/curated/properties.parquet')

filtered_df = df \
    .filter((F.col('beds') > 0) & (F.col('beds') <= 6)) \
    .filter((F.col('baths') > 0) & (F.col('baths') <= 6)) \
    .filter((F.col('parking') >= 0) & (F.col('parking') <= 6))

print("removing {} invalid rows".format(df.count() - filtered_df.count()))


removing 156 invalid rows


Saving the dataframe to parquet file

In [11]:
filtered_df.write.mode("overwrite").parquet("../data/curated/valid_properties.parquet")

MERGING:
The following section merges the parquet files with four datasets taken from https://dbr.abs.gov.au/region.html?lyr=ra&rgn=20 which are for major cities, inner regional, outer regional and remote australia inside victoria as well as population data taken from the 2021 census found on the abs website

In [12]:
# downloading datasets
import urllib.request as req
import zipfile
import os

# url = [major city, inner regional, outer regional, remote]
url = ['https://dbr.abs.gov.au/processedData/csv/RA_20.csv', 'https://dbr.abs.gov.au/processedData/csv/RA_21.csv','https://dbr.abs.gov.au/processedData/csv/RA_22.csv','https://dbr.abs.gov.au/processedData/csv/RA_23.csv']
url_header = ['major', 'inner_r', 'outer_r', 'remote']
urls = {}
for i,j in enumerate(url_header):
    req.urlretrieve(url[i], '../data/raw/{}.csv'.format(j))

zip_file_path = '../data/raw/census_population.zip'
extract_path = '../data/raw/census_data'
req.urlretrieve('https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_POA_for_VIC_short-header.zip', zip_file_path)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Use the extracted files
extracted_files = os.listdir(extract_path)
for file in extracted_files:
    if file.endswith('.csv'):
        csv_path = os.path.join(extract_path, file)
        with open(csv_path, 'r') as f:
            content = f.read()

os.remove(zip_file_path)

In [13]:
# converting csv files to spark dataframe
spark = SparkSession.builder.appName("merge").getOrCreate()

24/10/18 10:43:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


We convert the ABS csv files into a spark dataframe

In [14]:
dfs = []
url_header = ['major', 'inner_r', 'outer_r', 'remote']
region = ['metropolitan victoria', 'inner regional victoria', 'outer regional victoria', 'remote victoria']

i = 1
while i < 5:
    df = spark.read.csv('../data/raw/{}.csv'.format(url_header[i-1]), header=True, inferSchema=True)
    
    # only using columns from 2018-2022
    df = df.drop('2011', '2015', '2016', '2023')
    # assigning region values
    df = df.withColumn('region', F.lit(region[i-1]))
    dfs.append(df)
    i += 1

we merge our separate regional dataframes and combine them into one


In [15]:
from functools import reduce

# function that combines all dataframes into one merged_df
merged_df = reduce(lambda df1, df2: df1.union(df2), dfs) 

In [16]:
# output to parquet
merged_df.write.mode('overwrite').parquet('../data/raw/region_summary.parquet') 

In [17]:
# deleting unused files
files = ['../data/raw/major.csv', '../data/raw/outer_r.csv', '../data/raw/inner_r.csv', '../data/raw/remote.csv']

for file in files:
    if os.path.exists(file):
        os.remove(file)
        print(f"{file} has been deleted.")
    else:
        print(f"{file} does not exist.")

../data/raw/major.csv has been deleted.
../data/raw/outer_r.csv has been deleted.
../data/raw/inner_r.csv has been deleted.
../data/raw/remote.csv has been deleted.


=======================================================================================================================\
changing postcode dataset to be compatible with the properties dataset from Domain.com

This will enable us to merge the data into our property dataset. We will do this by linking our region names with the remoteness area names, using the australian postcode dataset by matthew proctor

In [19]:
url = "https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv"
output_path = "../data/raw/postcodes.csv"
req.urlretrieve(url, output_path)
df = spark.read.csv(output_path, header=True, inferSchema=True)

HTTPError: HTTP Error 403: Forbidden

Preprocessing the postcode data to only include features of interest i.e. postcode, locality, state, local government region and regional area name for merging

In [20]:
# only want postcode, locality, state and ra_2021_name columns
wanted_col = ['postcode', 'locality', 'state', 'lgaregion', 'RA_2021_NAME']
new_df = df.select(*wanted_col)
# only take rows from victoria
new_df = new_df.filter(new_df['state'] == 'VIC')
# remove the state column as its not important anymore
new_df = new_df.drop('state')

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `postcode` cannot be resolved. Did you mean one of the following? [`region`, `2018`, `2019`, `2020`, `2021`].;
'Project ['postcode, 'locality, 'state, 'lgaregion, 'RA_2021_NAME]
+- Project [Measure Code#562, Parent Description#563, Description#564, 2018#567, 2019#568, 2020#569, 2021#570, 2022#571, remote victoria AS region#593]
   +- Project [Measure Code#562, Parent Description#563, Description#564, 2018#567, 2019#568, 2020#569, 2021#570, 2022#571]
      +- Relation [Measure Code#562,Parent Description#563,Description#564,2011#565,2016#566,2018#567,2019#568,2020#569,2021#570,2022#571,2023#572] csv


reading the abs census data into spark dataframes

In [62]:
df1 = spark.read.csv('../data/raw/census_data/2021 Census GCP Postal Areas for VIC/2021Census_G01_VIC_POA.csv', header=True, inferSchema=True)
df2 = spark.read.csv('../data/raw/census_data/2021 Census GCP Postal Areas for VIC/2021Census_G02_VIC_POA.csv', header=True, inferSchema=True)

Renaming the columns into more consistent and readable names

In [63]:
df2 = df2 \
    .withColumnRenamed('Median_age_persons', 'median age') \
    .withColumnRenamed('Median_rent_weekly', 'median rent') \
    .withColumnRenamed('Median_tot_fam_inc_weekly', 'median family weekly income') \
    .withColumnRenamed('POA_CODE_2021', 'postcode')

df2.show()

+--------+----------+-----------------------------+---------------------------+-----------+---------------------------+----------------------------+-------------------------+----------------------+
|postcode|median age|Median_mortgage_repay_monthly|Median_tot_prsnl_inc_weekly|median rent|median family weekly income|Average_num_psns_per_bedroom|Median_tot_hhd_inc_weekly|Average_household_size|
+--------+----------+-----------------------------+---------------------------+-----------+---------------------------+----------------------------+-------------------------+----------------------+
| POA3000|        28|                         1733|                        768|        370|                       1857|                         1.1|                     1306|                   1.7|
| POA3002|        42|                         2383|                       1532|        480|                       3733|                         0.9|                     2345|                   1.8|
| POA3003|

In [64]:
# renaming the columns we want to keep
df1 = df1\
    .withColumnRenamed('POA_CODE_2021', 'postcode') \
    .withColumnRenamed('Tot_P_M', 'total male population - 2021') \
    .withColumnRenamed('Tot_P_F', 'total female population - 2021') \
    .withColumnRenamed('Tot_P_P', 'total population - 2021') \
    .withColumnRenamed('Australian_citizen_P', 'australian citizens')
df1.show()

24/10/17 21:27:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+----------------------------+------------------------------+-----------------------+------------+------------+------------+-------------+-------------+-------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+----------+----------+----------+---------------------------+---------------------------+---------------------------+-----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+---------------------

In [65]:
#preprocessing the census population dataset
cols = ['postcode', 'total male population - 2021', 'total female population - 2021', 'total population - 2021', 'australian citizens']
df1 = df1.withColumn("postcode", F.col('postcode').substr(4, 4))
df1 = df1.select(*cols)

#preprocessing the rental data census dataset
df2 = df2.withColumn("postcode", F.col('postcode').substr(4, 4))
cols = ['postcode', 'median rent', 'median family weekly income', 'median age']
df2 = df2.select(*cols)

Performing merges and saving dataframes

In [66]:
#merging the two census datasets
merge_df = df1.join(df2, on = 'postcode', how = 'left')
merged_df = new_df.join(merge_df, on = 'postcode', how = 'left')

In [67]:
merged_df.write.mode('overwrite').parquet('../data/raw/victorian_postcodes.parquet')
region_df = spark.read.parquet("../data/raw/region_summary.parquet")

Standardising for a seamless merge

In [70]:
region_df = region_df.withColumn(
    'region',
    F.when(F.col('region') == 'metropolitan victoria', 'Major Cities of Australia' )
    .otherwise(F.col('region'))).withColumn(
    'region',
    F.when(F.col('region') == 'inner regional victoria', 'Inner Regional Australia' )
    .otherwise(F.col('region'))
    ).withColumn(
    'region',
    F.when(F.col('region') == 'outer regional victoria', 'Outer Regional Australia' )
    .otherwise(F.col('region'))
    ).withColumn(
    'region',
    F.when(F.col('region') == 'remote victoria', 'Remote Australia' )
    .otherwise(F.col('region'))
    )
    

Getting male and female region population columns and merging them in

In [71]:
totalm_df = region_df.where(F.col('Measure Code') == 'ERP_M_20')
totalf_df = region_df.where(F.col('Measure Code') == 'ERP_F_20') 

In [72]:
totalm_df = totalm_df.withColumnRenamed('2022', 'total region male population - 2022')
totalf_df = totalf_df.withColumnRenamed('2022', 'total region female population - 2022')

In [73]:
col1 = ['region', 'total region male population - 2022']
col2 = ['region', 'total region female population - 2022']

totalm_df = totalm_df.select(*col1)
totalf_df = totalf_df.select(*col2)
totalm_df.show()

+--------------------+-----------------------------------+
|              region|total region male population - 2022|
+--------------------+-----------------------------------+
|Major Cities of A...|                          2538968.0|
|Inner Regional Au...|                           606425.0|
|Outer Regional Au...|                           128873.0|
|    Remote Australia|                             1744.0|
+--------------------+-----------------------------------+



In [74]:
merged_df = merged_df.withColumnRenamed('RA_2021_NAME', 'region')
merged_df1 = merged_df.join(totalm_df, on = 'region', how = 'left')
merged_df1 = merged_df1.join(totalf_df, on = 'region', how = 'left')

In [75]:
merged_df1 = merged_df1.drop('locality')
merged_df1.show()

+--------------------+--------+------------+----------------------------+------------------------------+-----------------------+-------------------+-----------+---------------------------+----------+-----------------------------------+-------------------------------------+
|              region|postcode|   lgaregion|total male population - 2021|total female population - 2021|total population - 2021|australian citizens|median rent|median family weekly income|median age|total region male population - 2022|total region female population - 2022|
+--------------------+--------+------------+----------------------------+------------------------------+-----------------------+-------------------+-----------+---------------------------+----------+-----------------------------------+-------------------------------------+
|Major Cities of A...|    3000|   Melbourne|                       21548|                         21539|                  43084|              14713|        370|                  

In [76]:
merged_df1.write.mode('overwrite').parquet('../data/raw/victorian_postcodes.parquet')

=====================================================================================\
now we are merging our population data with our property data

In [77]:
# reading dataframes
property_df = spark.read.parquet('../data/curated/valid_properties.parquet')
pop_df = spark.read.parquet('../data/raw/victorian_postcodes.parquet')

In [78]:
# merge on postcode
final_merge_df = property_df.join(pop_df, on = 'postcode', how = 'left')

In [79]:
final_merge_df.show()

+--------+-----+----------+---------+-------------+----+-----+-------+--------------------+---------------+----------------------------+------------------------------+-----------------------+-------------------+-----------+---------------------------+----------+-----------------------------------+-------------------------------------+
|postcode| cost|    suburb|furnished|property_type|beds|baths|parking|              region|      lgaregion|total male population - 2021|total female population - 2021|total population - 2021|australian citizens|median rent|median family weekly income|median age|total region male population - 2022|total region female population - 2022|
+--------+-----+----------+---------+-------------+----+-----+-------+--------------------+---------------+----------------------------+------------------------------+-----------------------+-------------------+-----------+---------------------------+----------+-----------------------------------+----------------------------

In [80]:
final_merge_df.write.mode('overwrite').parquet('../data/curated/merged_df.parquet')